In [1]:
from datasets import *
from transformers import *
from tokenizers import *
import os
import json

2024-06-16 22:06:50.618181: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


RuntimeError: Failed to import transformers.training_args_tf because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
dataset = load_dataset("cc_news", split="train")

In [ ]:
ds = dataset.train_test_split(test_size=0.1)
ds["train"], ds["test"]

In [ ]:
for t in ds["train"]["text"][:2]:
    print(t)
    print("="*20)

In [5]:
def dataset_to_text(dataset, output_filename="data.txt"):
    "Saving dataset to disk"
    with open(output_filename, "w") as f:
        for t in dataset["text"]:
            print(t, file=f)

In [6]:
dataset_to_text(ds["train"], "train.txt")
dataset_to_text(ds["test"], "test.txt")

### Third-party dataset

In [8]:
features = Features({
    'text': Value('string')  # Assuming each file contains plain text
})
custom_dataset = load_dataset('text', data_files={'train': '/home/jupyter/txt_tmp/*.txt'})

Resolving data files:   0%|          | 0/49 [00:00<?, ?it/s]

In [9]:
# custom_dataset = custom_dataset.train_test_split(test_size=0.001)
custom_dataset["train"]#, custom_dataset["test"]

Dataset({
    features: ['text'],
    num_rows: 2984
})

## Tokenization

tokenization params

In [7]:
special_tokens = [
    "[PAD]", 
    "[UNK]", 
    "[CLS]", 
    "[SEP]", 
    "[MASK]", 
    "<S>", 
    "<T>"
]

# files = ["txt_tmp/102390549_42b1a6b46dd56532086866d3c20f2581.rtf.txt"]
files = ["train.txt"]


vocab_size = 20_000
max_length = 256
truncate_longer_samples = True

# BERT params
# vocab_size = 30_000
# max_length = 512 # at least
# truncate_longer_samples = True

In [8]:
from tokenizers import BertWordPieceTokenizer

In [9]:
tokenizer = BertWordPieceTokenizer()
tokenizer.train(
    files=files, 
    vocab_size=vocab_size, 
    special_tokens=special_tokens,
)
tokenizer.enable_truncation(max_length=max_length)

In [10]:
!mkdir news_pre_trained_BERT

mkdir: cannot create directory ‘news_pre_trained_BERT’: File exists


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
pre_trained_bert_path = "news_pre_trained_BERT"

if not os.path.isdir(pre_trained_bert_path):
    os.mkdir(pre_trained_bert_path)

tokenizer.save_model(pre_trained_bert_path)

['news_pre_trained_BERT/vocab.txt']

In [12]:
with open(os.path.join(pre_trained_bert_path, "config.json"), "w") as f:
    tokenizer_cfg = {
        "do_lower_case": True,
        "unk_token": "[UNK]",
        "sep_token": "[SEP]",
        "pad_token": "[PAD]",
        "cls_token": "[CLS]",
        "mask_token": "[MASK]",
        "model_max_length": max_length,
        "max_len": max_length,
    }
    json.dump(tokenizer_cfg, f)

## Fast implementation

In [13]:
from transformers import BertTokenizerFast
new_tokenizer = BertTokenizerFast(
    tokenizer_object=tokenizer._tokenizer,
)

In [14]:
type(new_tokenizer)

transformers.models.bert.tokenization_bert_fast.BertTokenizerFast

In [15]:
new_tokenizer.save_pretrained("./saved_models/")

tokenizer config file saved in ./saved_models/tokenizer_config.json
Special tokens file saved in ./saved_models/special_tokens_map.json


('./saved_models/tokenizer_config.json',
 './saved_models/special_tokens_map.json',
 './saved_models/vocab.txt',
 './saved_models/added_tokens.json',
 './saved_models/tokenizer.json')

In [16]:
new_tokenizer.is_fast

True

In [17]:
new_tokenizer

BertTokenizerFast(name_or_path='', vocab_size=20000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<S>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("<T>", rs

In [18]:
tokenizer = BertTokenizerFast.from_pretrained(pre_trained_bert_path)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file news_pre_trained_BERT/config.json
loading configuration file news_pre_trained_BERT/config.json


In [19]:
tokenizer

BertTokenizerFast(name_or_path='news_pre_trained_BERT', vocab_size=20000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [20]:
def encode_with_truncation(examples):
    """Mapping fucntion to tokenize the sentences passed with truncation"""
    return tokenizer(
        examples["text"], 
        truncation=True, 
        padding="max_length", 
        max_length=max_length, 
        return_special_tokens_mask=True,
    )

In [21]:
def encode_without_truncation(examples):
    "Mapping function to tokenize the sentences passed without truncation"
    return tokenizer(
        examples["text"],
        return_special_tokens_mask=True,
    )

In [22]:
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation

In [23]:
# tokenizing train dataset
train_dataset = ds["train"].map(encode, batched=True)

#tokenizing the testing dataset
test_dataset = ds["test"].map(encode, batched=True)

Map:   0%|          | 0/637416 [00:00<?, ? examples/s]

In [24]:
if truncate_longer_samples:
    train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
    test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
    test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
    train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

In [25]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset
#     and generate chunks of max_seq_length. 

def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    
    # split by chunks of max_len
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    
    return result

In [26]:
if not truncate_longer_samples:
    train_dataset = train_dataset.map(group_texts, batched=True, desc=f"Grouping texts in chunks of {max_length}")
    test_dataset = test_dataset.map(group_texts, batched=True, desc=f"Grouping texts in chunks of {max_length}")
    
    train_dataset.set_format("torch")
    test_dataset.set_format("torch")

In [27]:
len(train_dataset), len(test_dataset)

(637416, 70825)

In [28]:
train_dataset

Dataset({
    features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 637416
})

In [30]:
train_dataset[0]

{'input_ids': tensor([    2, 13360,  1018, 13281,    19, 15091,    18,  7026,    45,  2189,
          2055,    45,  3566,  1587,  4671, 10819,  1585,  2941,    20,  2506,
            18,  2063,    18,  1564,  9999,  5750,    18,  6979,    20,  1552,
         10819,  1630,  3110,  1590,  1552,  4559,  1569,  1552,  3942,  1564,
          5584,    18,  6979,    20,    18,  1585,  2540,    20,    14,  2445,
            32,  6209,  2413,  3365,  1818,    21,  1983,    15,  5584,    18,
          6979,    20,   338, 11666,  8988,  1564,  3321,  1556,    20,    45,
          3087, 16045,  1738,  9301,    20,  2840,  3814,  3703,  8929,    20,
          5921, 17643,  2747,    20,  2967,    20,  8612,  4064,    19,  9248,
          1696,  6912,    20,  1552,  9376,  2182,  1569,  1874,  1739,  2738,
          1565, 12917,    45,  3546,  9160,  1569,  2447, 12471,  1564,  1672,
          4654,  3442,  3156,  6030,  1565,  1738,  1036, 11074,  1552,  4897,
            19,  1689,  1604,    20,  1

## BERT classical config
- vocab_size=30522
- hidden_size=768
- num_hidden_layers=12
- num_attention_heads=12
- intermediate_size=3072
- hidden_act='gelu'
- hidden_dropout_prob=0.1
- attention_probs_dropout_prob=0.1
- max_position_embeddings=512
- type_vocab_size=2
- initializer_range=0.02
- layer_norm_eps=1e-12
- pad_token_id=0
- gradient_checkpointing=False

In [31]:
model_config = BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)

In [34]:
from transformers import BertModel, BertConfig

In [36]:
configuration = BertConfig()
model = BertModel(configuration)
configuration = model.config

In [37]:
model_config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 256,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.40.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 20000
}

In [38]:
model = BertForMaskedLM(config=model_config)

Generate config GenerationConfig {
  "pad_token_id": 0
}



In [39]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(20000, 768, padding_idx=0)
      (position_embeddings): Embedding(256, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [40]:
# initialize the data collator, randomly masking 20%(def is 15%) of the tokens for the Masked language
# modeling (MLM) task

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=True, 
    mlm_probability=0.2, 
    return_tensors='pt',
)

In [41]:
data_collator

DataCollatorForLanguageModeling(tokenizer=BertTokenizerFast(name_or_path='news_pre_trained_BERT', vocab_size=20000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, mlm=True, mlm_probability=0.2, pad_to_multiple_of=None, tf_e

In [43]:
%pip install accelerate # this used for multiple GPUs

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


### BERT pre-training parameters

In [53]:
training_args = TrainingArguments(
    output_dir=pre_trained_bert_path, # output directory to where save model checkpoint 
    evaluation_strategy="steps",      # evaluate each 'logging_steps' steps
    overwrite_output_dir=True,
    num_train_epochs=10,              # number of training epochs (feel free to tweak)
    per_device_train_batch_size=32,   # 16, 32 the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,    # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,    # evaluation batch size
    logging_steps=100,                 # evaluate, log and save model checkpoints evenry 1000 steps
    # auto_find_batch_size=True,      # find the optimal batch size for your GPU
    load_best_model_at_end=True,      # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,             # whether you dont have much space so you let only 3 model weights saved in the disk
)

using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [48]:
!pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [52]:
training_args

<module 'transformers.training_args' from '/opt/conda/lib/python3.10/site-packages/transformers/training_args.py'>

In [54]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

AttributeError: module 'transformers.training_args' has no attribute 'full_determinism'